In [26]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, applications, callbacks
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Removido data augmentation nesta etapa
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import random

In [15]:
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)

def plot_confusion(cm, labels, title=None):
    disp = ConfusionMatrixDisplay(cm, display_labels=labels)
    disp.plot(cmap='Blues', xticks_rotation=45)
    if title:
        plt.title(title)
    plt.show()

In [16]:
def load_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalização ao intervalo [0,1]
    x_train = x_train.astype('float32') / 255.0
    x_test  = x_test.astype('float32') / 255.0
    # One-hot para treino/fine-tuning
    y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
    y_test_cat  = tf.keras.utils.to_categorical(y_test, 10)
    # Labels achatadas para classifier raso
    y_train_lbl = y_train.flatten()
    y_test_lbl  = y_test.flatten()
    return x_train, y_train_cat, y_train_lbl, x_test, y_test_cat, y_test_lbl

In [17]:
def get_cifar10_dataset(batch_size=64, buffer_size=5000):
    # Carrega dados
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Normalização
    x_train = x_train.astype('float32') / 255.0
    x_test  = x_test.astype('float32') / 255.0
    # One-hot labels
    y_train = tf.keras.utils.to_categorical(y_train, 10).astype('float32')
    y_test  = tf.keras.utils.to_categorical(y_test, 10).astype('float32')

    # Cria datasets
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_ds = train_ds.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return train_ds, test_ds

In [18]:
def build_cnn_scratch(input_shape=(32, 32, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)

In [19]:
def train_cnn_scratch():
    # Carrega arrays numpy e datasets
    x_train, y_train_cat, _, x_test, y_test_cat, _ = load_data()
    train_ds, test_ds = get_cifar10_dataset()

    # Constrói e compila o modelo
    model = build_cnn_scratch()
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Treina usando tf.data
    history = model.fit(
        train_ds,
        epochs=30,
        validation_data=test_ds,
        callbacks=[early],
        verbose=2
    )

    # Avaliação
    loss, acc = model.evaluate(test_ds, verbose=0)
    print(f"CNN do zero: Test accuracy = {acc:.4f}")

    # Matriz de Confusão via arrays numpy (evita iterator esgotado)
    y_true = y_test_cat.argmax(axis=1)
    y_pred = model.predict(x_test).argmax(axis=1)
    class_names = ['airplane','automobile','bird','cat','deer',
                   'dog','frog','horse','ship','truck']
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion(cm, class_names, title='Confusion - CNN from Scratch')

    return model

In [33]:
def build_finetune_model(backbone='ResNet50',
                         input_shape=(32,32,3),
                         num_classes=10,
                         unfreeze_from=140):
    # carrega a base pré-treinada sem o top
    if backbone == 'VGG16':
        base = applications.VGG16(
            weights='imagenet',
            include_top=False,
            input_shape=input_shape
    )
    elif backbone == 'MobileNetV2':
        base = applications.MobileNetV2(
            weights='imagenet', include_top=False,
            input_shape=input_shape)
    else:
        base = applications.ResNet50(
            weights='imagenet', include_top=False,
            input_shape=input_shape)

    # congela tudo
    for layer in base.layers:
        layer.trainable = False
    # descongela só as últimas
    for layer in base.layers[unfreeze_from:]:
        layer.trainable = True

    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu',
                 kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs=base.input, outputs=outputs)

In [32]:
def train_fine_tuning(x_train, y_train_cat, x_test, y_test_cat,
                      backbone='ResNet50',
                      unfreeze_from=140,
                      epochs=20,
                      batch_size=64):
    
    model = build_finetune_model(backbone, unfreeze_from=unfreeze_from)
    opt = Adam(learning_rate=1e-4)

    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    # early = callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
    early = callbacks.EarlyStopping(monitor='val_accuracy', patience=13, restore_best_weights=True)
    
    
    history = model.fit(
        x_train, y_train_cat,
        validation_data=(x_test, y_test_cat),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[early],
        verbose=2
    )
    
    loss, acc = model.evaluate(x_test, y_test_cat, verbose=0)
    print(f"Fine-tune {backbone}: Test accuracy = {acc:.4f}")

    # plot matriz de confusão
    y_pred = model.predict(x_test).argmax(axis=1)
    y_true = y_test_cat.argmax(axis=1)
    class_names = ['airplane','automobile','bird','cat','deer',
                   'dog','frog','horse','ship','truck']
    cm = confusion_matrix(y_true, y_pred)
    plot_confusion(cm, class_names,
                   title=f"Confusion – Fine-tune {backbone}")

    return model

In [ ]:
# Treina apenas a CNN do zero com tf.data.Dataset
train_cnn_scratch()

In [ ]:
x_train, y_train_cat, _, x_test, y_test_cat, _ = load_data()
ft_model = train_fine_tuning(
    x_train, y_train_cat, x_test, y_test_cat,
    backbone='ResNet50',
    unfreeze_from=160,
    epochs=30,
    batch_size=64
)

In [ ]:
x_train, y_train_cat, _, x_test, y_test_cat, _ = load_data()
ft_model = train_fine_tuning(
    x_train, y_train_cat, x_test, y_test_cat,
    backbone='VGG16',
    unfreeze_from=160,
    epochs=30,
    batch_size=64
)

In [ ]:
x_train, y_train_cat, _, x_test, y_test_cat, _ = load_data()
ft_model = train_fine_tuning(
    x_train, y_train_cat, x_test, y_test_cat,
    backbone='MobileNetV2',
    unfreeze_from=160,
    epochs=30,
    batch_size=64
)